In [ ]:
# Import required libraries
import pickle
import copy
import pathlib
import dash
import math
import datetime as dt
import pandas as pd
from dash.dependencies import Input, Output, State, ClientsideFunction
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

from jupyter_dash import JupyterDash

In [ ]:
sdmx_url = 'https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.{}....?format=csv'

In [ ]:
data.head()

In [ ]:
indicators_dict = {
    'EDU_SDG_STU_L2_MATH': {
        'name': 'Proportion of children at the end of lower secondary education reaching minimum proficiency in math', 
        'compare': ['EDU_SDG_STU_L2_READING', 'EDU_SDG_STU_L1_GLAST_MATH', 'EDU_SDG_STU_L1_G2OR3_MATH', 'EDUNF_NERA_L2',]
    },
    'EDU_SDG_STU_L2_READING': {
        'name': 'Proportion of children at the end of lower secondary education reaching minimum proficiency in reading', 
        'compare': ['EDUNF_ROFST_L2', 'EDU_SDG_STU_L2_MATH', 'EDU_SDG_STU_L1_GLAST_READING', 'EDU_SDG_STU_L1_G2OR3_READING', 'EDUNF_NERA_L2']
    }
}
data = pd.DataFrame()
inds = []
for key,value in indicators_dict.items():
    inds.append(key)
    inds.extend(value['compare'])
    

for ind in inds:
    sdmx = pd.read_csv(sdmx_url.format(ind))
    sdmx['CODE'] = ind        
    data = data.append(sdmx)

In [69]:
data[data['CODE'] == 'EDUNF_ROFST_L2'] 

,Geographic area,Indicator,Sex,Age,Residence,Wealth Quintile,TIME_PERIOD,OBS_VALUE,Unit of measure,OBS_FOOTNOTE,Frequency,DATA_SOURCE,Unit multiplier,OBS_STATUS,CODE
0,Albania,Out of school children rate: Lower secondary e...,Total,School-age population,Total,Total,2000,7.12167,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
1,Albania,Out of school children rate: Lower secondary e...,Total,School-age population,Total,Total,2001,6.31571,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
2,Albania,Out of school children rate: Lower secondary e...,Total,School-age population,Total,Total,2013,5.42662,%,NaN,Annual,UIS,Units,E,EDUNF_ROFST_L2
3,Albania,Out of school children rate: Lower secondary e...,Total,School-age population,Total,Total,2014,5.24218,%,NaN,Annual,UIS,Units,E,EDUNF_ROFST_L2
4,Albania,Out of school children rate: Lower secondary e...,Total,School-age population,Total,Total,2015,3.05841,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,Uzbekistan,Out of school children rate: Lower secondary e...,Male,School-age population,Total,Total,2014,6.09852,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
906,Uzbekistan,Out of school children rate: Lower secondary e...,Male,School-age population,Total,Total,2015,6.20280,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
907,Uzbekistan,Out of school children rate: Lower secondary e...,Male,School-age population,Total,Total,2016,4.39841,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2
908,Uzbekistan,Out of school children rate: Lower secondary e...,Male,School-age population,Total,Total,2017,3.39600,%,NaN,Annual,UIS,Units,A,EDUNF_ROFST_L2


In [ ]:
# Create controls
county_options = [
    {"label": str(country), "value": str(country)} for country in data['Geographic area'].unique()
]

In [ ]:
county_options

In [ ]:
# Create global chart template
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNrOWJqb2F4djBnMjEzbG50amg0dnJieG4ifQ.Zme1-Uzoi75IaFbieBDl3A"

layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
    title="Satellite Overview",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style="light",
        center=dict(lon=-78.05, lat=42.54),
        zoom=7,
    ),
)

In [62]:
years =[i for i in range(2010, 2020)]

indicators = data['Indicator'].unique()

{index: str(year) for index,year in enumerate(years)}

{0: '2010',
 1: '2011',
 2: '2012',
 3: '2013',
 4: '2014',
 5: '2015',
 6: '2016',
 7: '2017',
 8: '2018',
 9: '2019'}

In [66]:
# Build App
app = JupyterDash(__name__)

# Create app layout
app.layout = html.Div(
    [
        dcc.Store(id="aggregate_data"),
        # empty Div to trigger javascript file for graph resizing
        html.Div(id="output-clientside"),
        html.Div(
            [
                html.Div(
                    [
                        html.Img(
                            src='UNICEFLogo.png',
                            id="unicef-logo",
                            style={
                                "height": "60px",
                                "width": "auto",
                                "margin-bottom": "25px",
                            },
                        )
                    ],
                    className="one-third column",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.H3(
                                    "Education Dashboard",
                                    style={"margin-bottom": "0px"},
                                ),
                                html.H5(
                                    "4.1 Overview", style={"margin-top": "0px"}
                                ),
                            ]
                        )
                    ],
                    className="one-half column",
                    id="title",
                ),
                html.Div(
                    [
                        html.A(
                            html.Button("Learn More", id="learn-more-button"),
                            href="https://plot.ly/dash/pricing/",
                        )
                    ],
                    className="one-third column",
                    id="button",
                ),
            ],
            id="header",
            className="row flex-display",
            style={"margin-bottom": "25px"},
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.P(
                            "Filter by year:",
                            className="control_label",
                        ),
                        dcc.RangeSlider(
                            id="year_slider",
                            min=0,
                            max=len(years),
                            step=None,
                            marks={index: str(year) for index,year in enumerate(years)},
                            value=[0, len(years)],
                            className="dcc_control",
                        ),
                        html.P("Filter by Country:", className="control_label"),
                        dcc.Dropdown(
                            id="country_selector",
                            options=county_options,
                            multi=True,
                            value=[item['value'] for item in county_options],
                            className="dcc_control",
                        ),
                    ],
                    className="pretty_container four columns",
                    id="cross-filter-options",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.Div(
                                    [html.H6(id="well_text"), html.P("No. of Wells")],
                                    id="wells",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="gasText"), html.P("Gas")],
                                    id="gas",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="oilText"), html.P("Oil")],
                                    id="oil",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="waterText"), html.P("Water")],
                                    id="water",
                                    className="mini_container",
                                ),
                            ],
                            id="info-container",
                            className="row container-display",
                        ),
                        html.Div(
                            [dcc.Graph(id="count_graph")],
                            id="countGraphContainer",
                            className="pretty_container",
                        ),
                        
                    ],
                    id="right-column",
                    className="eight columns",
                ),
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.Div(
                            [dcc.Graph(id="maths_graph")],
                            className="pretty_container",
                        ),
                        dcc.Dropdown(
                            id='maths-xaxis-column',
                            options=[
                                {'label': item['Indicator'], 'value': item['CODE']} for item in 
                                data[
                                    data['CODE'].isin(indicators_dict['EDU_SDG_STU_L2_MATH']['compare'])
                                ][['CODE', 'Indicator']].drop_duplicates().to_dict('records')
                            ],
                            multi=True,
#                             value=[item for item in indicators],
                            className="dcc_control",
                        ),
                    ],
                    className="six columns",
                ),
                html.Div(
                    [
                        html.Div(
                            [dcc.Graph(id="reading_graph")],
                            className="pretty_container",
                        ),
                        dcc.Dropdown(
                            id='reading-xaxis-column',
                            options=[
                                {'label': item['Indicator'], 'value': item['CODE']} for item in 
                                data[
                                    data['CODE'].isin(indicators_dict['EDU_SDG_STU_L2_READING']['compare'])
                                ][['CODE', 'Indicator']].drop_duplicates().to_dict('records')
                            ],                            multi=True,
#                             value=[item for item in indicators],
                            className="dcc_control",
                        ),
                    ],
                    className="six columns",
                )
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [dcc.Graph(id="pie_graph")],
                    className="pretty_container seven columns",
                ),
                html.Div(
                    [dcc.Graph(id="aggregate_graph")],
                    className="pretty_container five columns",
                ),
            ],
            className="row flex-display",
        ),
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)

# Create callbacks
app.clientside_callback(
    ClientsideFunction(namespace="clientside", function_name="resize"),
    Output("output-clientside", "children"),
    [Input("count_graph", "figure")],
)

# Slider -> count graph
@app.callback(Output("year_slider", "value"), [Input("count_graph", "selectedData")])
def update_year_slider(count_graph_selected):

    if count_graph_selected is None:
        return [years[0], years[-1]]

    nums = [int(point["pointNumber"]) for point in count_graph_selected["points"]]
    return [min(nums) + years[0], max(nums) + years[-1]]

# Selectors -> maths graph
@app.callback(
    Output("maths_graph", "figure"),
    [
        Input("year_slider", "value"),
        Input("maths-xaxis-column", "value"),
    ],
#     [State("lock_selector", "value"), State("count_graph", "relayoutData")],
)
def make_maths_figure(year_slider, xaxis):
    indicator = 'EDU_SDG_STU_L2_MATH'
    dff = data
    fig = go.Figure()
    fig.add_trace(
            go.Bar(
                x=years,
                y=data[data['CODE'] == indicator]['OBS_VALUE'],
                name=indicators_dict[indicator]['name']
            ),
        )
    if xaxis:
        for value in xaxis:
            fig.add_trace(
                go.Bar(
                    x=years,
                    y=data[data['CODE'] == value]['OBS_VALUE'],
                    name=data[data['CODE'] == value]['Indicator'].unique()[0]
                ),
            )
    fig.update_layout(
        title='Maths',
#         xaxis_tickfont_size=14,
        yaxis=dict(
            title='Proportion in %',
#             titlefont_size=16,
#             tickfont_size=14,
        ),
        legend=dict(
            orientation="h",
            y=-0.2
        ),
        barmode='group',
#         bargap=0.15, # gap between bars of adjacent location coordinates.
#         bargroupgap=0.1 # gap between bars of the same location coordinate.
    )

    
#     figure = dict(data=traces, layout=layout)
    return fig


# Selectors -> reading graph
@app.callback(
    Output("reading_graph", "figure"),
    [
        Input("year_slider", "value"),
        Input("reading-xaxis-column", "value"),
    ],
#     [State("lock_selector", "value"), State("count_graph", "relayoutData")],
)
def make_reading_figure(year_slider, xaxis):
    indicator = 'EDU_SDG_STU_L2_READING'
    dff = data
    fig = go.Figure()
    fig.add_trace(
            go.Scatter(
                mode="lines+markers",
                name=indicators_dict[indicator]['name'],
                x=years,
                y=data[data['CODE'] == indicator]['OBS_VALUE'],
                line=dict(shape="spline", smoothing=1.3, width=1, color="#fac1b7"),
                marker=dict(symbol="diamond-open"),
            ),
        )
    if xaxis:
        for value in xaxis:
            fig.add_trace(
                go.Scatter(
                    mode="lines+markers",
                        name=data[data['CODE'] == value]['Indicator'].unique()[0],
                        x=years,
                        y=data[data['CODE'] == value]['OBS_VALUE'],
                        line=dict(shape="spline", smoothing=1.3, width=1),
                        marker=dict(symbol="diamond-open"),
                ),
            )
    fig.update_layout(
        title='Reading',
#         xaxis_tickfont_size=14,
        yaxis=dict(
            title='Proportion in %',
#             titlefont_size=16,
#             tickfont_size=14,
        ),
        legend=dict(
            orientation="h",
            y=-0.2
        ),
#         barmode='group',
#         bargap=0.15, # gap between bars of adjacent location coordinates.
#         bargroupgap=0.1 # gap between bars of the same location coordinate.
    )

    
#     figure = dict(data=traces, layout=layout)
    return fig

# Run app and display result inline in the notebook
app.run_server()

Dash app running on http://127.0.0.1:8050/
